In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [5]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_disposition', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_period', 'koi_fpflag_co', 'koi_prad', 'koi_steff', 'koi_duration', 
                        'koi_time0bk', 'koi_kepmag', 'koi_model_snr', 'koi_duration_err2', 'koi_duration_err1', 'koi_prad_err1', 'koi_steff_err1', 
                        'koi_fpflag_ec', 'koi_prad_err2', 'koi_steff_err2', 'koi_time0bk_err1', 'koi_depth', 'koi_time0bk_err2', 'koi_period_err2', 
                        'koi_insol_err1', 'koi_impact', 'koi_period_err1', 'koi_insol_err2', 'koi_insol']]

# Create a Train Test Split

Use `koi_disposition` for the y values

In [6]:
X = selected_features.drop("koi_disposition", axis=1)
y = selected_features[['koi_disposition']]
print(X.shape, y.shape)

(6991, 26) (6991, 1)


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_period,koi_fpflag_co,koi_prad,koi_steff,koi_duration,koi_time0bk,koi_kepmag,koi_model_snr,...,koi_steff_err2,koi_time0bk_err1,koi_depth,koi_time0bk_err2,koi_period_err2,koi_insol_err1,koi_impact,koi_period_err1,koi_insol_err2,koi_insol
6122,0,0,6.768901,0,1.24,5737,3.61600,133.077240,14.725,10.8,...,-171,0.008440,123.1,-0.008440,-7.380000e-05,204.89,0.150,7.380000e-05,-103.87,253.30
6370,0,1,0.733726,0,0.86,5855,2.30900,132.020050,15.770,13.8,...,-175,0.007950,114.6,-0.007950,-6.060000e-06,2253.61,0.291,6.060000e-06,-677.78,2891.64
2879,1,0,7.652707,0,3.21,6328,79.89690,134.460380,13.099,254.3,...,-189,0.006190,641.1,-0.006190,-6.540000e-05,195.16,0.970,6.540000e-05,-64.34,226.81
107,0,0,7.953547,0,2.25,4768,2.63120,174.662240,15.660,38.4,...,-85,0.001820,875.4,-0.001820,-1.910000e-05,7.15,0.300,1.910000e-05,-10.12,55.37
29,0,0,4.959319,0,12.21,5712,2.22739,172.258529,15.263,696.5,...,-77,0.000083,9802.0,-0.000083,-5.150000e-07,146.52,0.831,5.150000e-07,-93.21,349.40


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [8]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
y_train_encoded =label_encoder.transform(y_train)
y_test_encoded = label_encoder.transform(y_test)


# Scale your data
X_scaler = MinMaxScaler().fit(X)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

/Users/bulu/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/bulu/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# Train the Model



In [9]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC 
model = SVC(kernel='linear')
model.fit(X_train_scaled, y_train_encoded)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [10]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train_encoded)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test_encoded)}")

Training Data Score: 0.8367346938775511
Testing Data Score: 0.8352402745995423


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [11]:
# Create the GridSearchCV model
# from sklearn.svm import SVC 
# model2 = SVC(kernel='linear')

from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10, 50],
              'max_iter': [500, 1000, 5000]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [12]:
# Train the model with GridSearch
grid.fit(X_train_scaled, y_train_encoded)

/Users/bulu/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/bulu/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:241: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] C=1, max_iter=500 ...............................................
[CV] ................... C=1, max_iter=500, score=0.766, total=   0.1s
[CV] C=1, max_iter=500 ...............................................


/Users/bulu/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:241: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s
/Users/bulu/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:241: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


[CV] ................... C=1, max_iter=500, score=0.759, total=   0.1s
[CV] C=1, max_iter=500 ...............................................
[CV] ................... C=1, max_iter=500, score=0.772, total=   0.1s
[CV] C=1, max_iter=1000 ..............................................
[CV] .................. C=1, max_iter=1000, score=0.831, total=   0.2s
[CV] C=1, max_iter=1000 ..............................................
[CV] .................. C=1, max_iter=1000, score=0.848, total=   0.2s
[CV] C=1, max_iter=1000 ..............................................


/Users/bulu/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:241: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


...

[CV] ................. C=50, max_iter=5000, score=0.879, total=   0.2s
[CV] C=50, max_iter=5000 .............................................
[CV] ................. C=50, max_iter=5000, score=0.883, total=   0.2s
[CV] C=50, max_iter=5000 .............................................


/Users/bulu/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:241: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:    5.9s finished


[CV] ................. C=50, max_iter=5000, score=0.860, total=   0.2s


/Users/bulu/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:241: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='linear',
                           max_iter=-1, probability=False, random_state=None,
                           shrinking=True, tol=0.001, verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [1, 5, 10, 50], 'max_iter': [500, 1000, 5000]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [13]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 50, 'max_iter': 5000}
0.8737364104520313


# Save the Model

In [14]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash

# import joblib
# filename = 'TylerNguyen.sav'
# joblib.dump(model, filename)